In [3]:
import pandas as pd 
import os
from tqdm import tqdm
from collections import defaultdict
import re
tqdm.pandas()


In [ ]:
DATA_PATH = "/Users/shasankgedda/Desktop/Research24/CodeMixedQAC/datasets/msmacroweb"
df = pd.read_csv("/Users/shasankgedda/Desktop/Research24/CodeMixedQAC/datasets/msmacroweb/queries_train.tsv",sep="\t",header=None,usecols=[1,2])
df.columns = ['query','query_locale']
df.head(3)

,query,query_locale
0,! [new branch] master -> origin/master (unable...,"en-US, zh-CN"
1,!doctype htm,"en-IN, en-US, en-WW, es-MX, ja-JP, ko-KR..."
2,!so command nightbot,"en-NZ, en-US, en-XA, en-AU, en-CA, en-PH, e..."


In [12]:
df['query'].nunique()

9206107

In [18]:
jp = pd.DataFrame(df[df['query_locale']=='ja-JP']['query'].value_counts()).reset_index()
jp.to_csv(os.path.join(DATA_PATH,"queries_train_ms_jp.csv"),index=None)
print(jp.shape)
es = pd.DataFrame(df[df['query_locale']=='es-ES']['query'].value_counts()).reset_index()
es.to_csv(os.path.join(DATA_PATH,"queries_train-ms_es.csv"),index=None)
print(es.shape)

(944911, 2)
(37230, 2)


In [19]:
jp.head(3)

,query,count
0,gherkin ブラウザ 新しいタブ,1
1,ｐｆａライニング管 小径,1
2,ｐｍ2.5nhk,1


In [20]:
es.head(3)

,query,count
0,manual ws-6979,1
1,wehbeonline.com,1
2,wohnelec catalogo,1


In [21]:
#df.groupby(['query_locale']).nunique()

In [29]:
# Run langauge detector 
import fasttext

# Load the pre-trained language detection model


# Class
class LanguageDetector:
    def __init__(self):
        model_path = "../../models/fasttext/lid.176.bin"
        self.model = fasttext.load_model(model_path)
        print(self.model)
        self.k = 3
    
    def predict_lang(self,text):
        predictions = self.model.predict(text,self.k)
        return predictions


In [30]:
lang_detect = LanguageDetector()
print(lang_detect)
lang_l = lang_detect.predict_lang("hej")
lang_l 

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

In [25]:
lang_detect.predict_lang("本気汁 アダルトdvd")[0]

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

In [66]:
lang_detect.predict_lang("butterfly バタフライ")[1]


array([0.93229467, 0.02319806, 0.02129641])

In [ ]:
#lang_detect.predict("butterfly (バタフライ)", k=3)


AttributeError: 'LanguageDetector' object has no attribute 'predict'

In [99]:
s = "butterfly バタフライ"
#re.search('[a-zA-Z]', s)
len(re.findall('[a-zA-Z]', s))

9

In [ ]:
#model.predict("qwatch ipアドレス", k=3)

(('__label__ja', '__label__en', '__label__es'),
 array([0.99296778, 0.0031089 , 0.0017918 ]))

### Run language detector on a dataset

In [87]:
pred_dict = defaultdict()
queries = jp['query'].tolist()

for query in tqdm(queries):
    lang,scores = lang_detect.predict_lang(query)
    #print(lang,scores)
    pred_dict[query] = {'lang':lang,'scores':scores}


100%|██████████| 12687/12687 [00:00<00:00, 108919.88it/s]


In [129]:
# For spanish
pred_dict = defaultdict()
queries = es['query'].tolist()

for query in tqdm(queries):
    lang,scores = lang_detect.predict_lang(query)
    #print(lang,scores)
    pred_dict[query] = {'lang':lang,'scores':scores}

100%|██████████| 10624/10624 [00:00<00:00, 84488.57it/s]


In [111]:
'__label__ja' in pred_dict['脚立']['lang']

#pred_dict

True

In [ ]:
### Apply thresholds

## Collect english code mixed - "__label__en" or # check english characters
en_jp_mix = defaultdict()

for i,(query,v) in enumerate(pred_dict.items()):
    #print(query,v)

    # Check if language label is japanese detected and number of english charcters >=3
    num_en_chars = len(re.findall('[a-zA-Z]', query))

    # For japanese 
    # if num_en_chars >=3 and '__label__ja' in v['lang']:
    #     print(query,v)
    #     #en_jp_mix.append(query)
    #     en_jp_mix[query] = {'lang':v['lang'],'scores':v['scores']}

    # For es 
    # detect english language
    if '__label__en' in v['lang'] and '__label__ja' in v['lang']:
        print(query,v)
        #en_jp_mix.append(query)
        en_jp_mix[query] = {'lang':v['lang'],'scores':v['scores']}


print(len(en_jp_mix))

ubereats 配達リュック ロゴなし {'lang': ('__label__ja', '__label__zh', '__label__en'), 'scores': array([9.99185205e-01, 4.29656560e-04, 2.61165143e-04])}
kaf046a41 枠なし {'lang': ('__label__ja', '__label__ar', '__label__uk'), 'scores': array([0.50558776, 0.1278602 , 0.12566382])}
stand by me ドラえもん2 dvd(特典なし) {'lang': ('__label__ja', '__label__en', '__label__ar'), 'scores': array([0.8967604 , 0.03411759, 0.03042005])}
nato ベルト 20mm {'lang': ('__label__ja', '__label__ko', '__label__en'), 'scores': array([0.99343562, 0.00282374, 0.00164854])}
iphone8 ケース {'lang': ('__label__ja', '__label__zh', '__label__ru'), 'scores': array([9.99423563e-01, 4.54549299e-04, 8.49145508e-05])}
lolサプライズ {'lang': ('__label__ja', '__label__zh', '__label__ru'), 'scores': array([9.98858869e-01, 9.70246270e-04, 1.70112849e-04])}
スマホカバーsh-01l ヒョウ柄 {'lang': ('__label__ja', '__label__en', '__label__ru'), 'scores': array([0.99007905, 0.0059009 , 0.00144697])}
lanハブ2ポート {'lang': ('__label__ja', '__label__zh', '__label__en'), 'sco

In [122]:
list(en_jp_mix.keys())

['ubereats 配達リュック ロゴなし',
 'kaf046a41 枠なし',
 'stand by me ドラえもん2 dvd(特典なし)',
 'nato ベルト 20mm',
 'iphone8 ケース',
 'lolサプライズ',
 'スマホカバーsh-01l ヒョウ柄',
 'lanハブ2ポート',
 'スマホケース iphonex',
 'macbook ハブ anker',
 'signstek / 縄なし縄跳び エクササイズ エアなわとび',
 'mctオイルcocolab',
 'perfk 食器',
 'epson インクカートリッジ 50',
 'dvd-r dl 50枚',
 'dhc 洗顔',
 'galaxy note 10 plus フィルム',
 'champion リュック',
 'dad フロアマット',
 'deep purple',
 'dentex フロス',
 'iphone11カバー',
 'iphone8 ケース mynus',
 'iphonexr ケース 韓国',
 'iphoneてんあーる 手帳型',
 'jnlせんユニット',
 'kindle端末',
 'iphone11さらさらフィルム',
 'iphone 11 pro ケース',
 'hdmi - vga 変換',
 'gucci マフラー メンズ',
 'happy belly ハッピーベリー',
 'ipad mini2 ガラスフィルム',
 'rizap 食品',
 'イノマタ化学 inomata-k',
 'switch ジョイコン',
 'switch マーベル',
 't300rs gt edition',
 'tyrolia グローブ',
 'usb 電球',
 'usb type c 外付けhdd',
 'usb マイクロsd',
 'サッカーボール amazon',
 'so-02g キャップ',
 'サーモス 水筒交換部品 jnr',
 'sh-01l ケース デニム',
 'seiko 腕時計',
 'sdカード 128gb',
 'usb電圧計',
 'クリアウォーターcnx',
 'yzfr6バックステップ',
 'windows10 パッケージ版',
 'windows10 pro アップグレード',
 'windows

In [126]:
jp_codemix = pd.DataFrame(list(en_jp_mix.keys()),columns=['query'])
jp_codemix.to_csv(os.path.join(DATA_PATH,'en_jp_cmix_list.csv'),index=None)

In [128]:
import pickle 
with open(os.path.join(DATA_PATH,'en_jp_cmix_dict.pkl'), 'wb') as f:
    pickle.dump(en_jp_mix, f)